In [46]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

In [3]:
%%time
dataset = load_dataset("lucasmccabe-lmi/CodeAlpaca-20k", split="train")


Generating train split: 100%|██████████| 20022/20022 [00:00<00:00, 572324.75 examples/s]

CPU times: user 219 ms, sys: 12.1 ms, total: 231 ms
Wall time: 10.7 s


In [20]:
dataset[0]

{'instruction': 'Create a function that takes a specific input and produces a specific output using any mathematical operators. Write corresponding code in Python.',
 'input': '',
 'output': 'def f(x):\n    """\n    Takes a specific input and produces a specific output using any mathematical operators\n    """\n    return x**2 + 3*x'}

In [4]:
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")


config.json: 100%|██████████| 644/644 [00:00<00:00, 172kB/s]
pytorch_model.bin: 100%|██████████| 663M/663M [00:43<00:00, 15.1MB/s] 
tokenizer_config.json: 100%|██████████| 685/685 [00:00<00:00, 490kB/s]
vocab.json: 100%|██████████| 899k/899k [00:01<00:00, 827kB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 573kB/s]
special_tokens_map.json: 100%|██████████| 441/441 [00:00<00:00, 240kB/s]


In [21]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"### Question: {example['instruction'][i]}\n ### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts


In [33]:
sample = (formatting_prompts_func(dataset[:2]))

In [34]:
sample[0]

'### Question: Create a function that takes a specific input and produces a specific output using any mathematical operators. Write corresponding code in Python.\n ### Answer: def f(x):\n    """\n    Takes a specific input and produces a specific output using any mathematical operators\n    """\n    return x**2 + 3*x'

In [49]:
response_template = " ### This is my answerr:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

args = transformers.TrainingArguments(
    num_train_epochs=3,
    output_dir="./output",
)
trainer = SFTTrainer(
    model,
    train_dataset=dataset.select(range(5)),
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    args=args
)

In [50]:

trainer.train()

100%|██████████| 3/3 [00:01<00:00,  2.77it/s]

{'train_runtime': 1.0869, 'train_samples_per_second': 13.801, 'train_steps_per_second': 2.76, 'train_loss': 0.0, 'epoch': 3.0}


TrainOutput(global_step=3, training_loss=0.0, metrics={'train_runtime': 1.0869, 'train_samples_per_second': 13.801, 'train_steps_per_second': 2.76, 'train_loss': 0.0, 'epoch': 3.0})

In [51]:
prompt = "Help me do hello world"
model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
generated_text = model.generate(**model_inputs, max_length=50, do_sample=True)

In [56]:
tokenizer.batch_decode(generated_text, skip_special_tokens=True)

['Help me do hello world, sorry but I can\'t read that...\nJust check your mail! (I believe you were in "emergency mail," which means it was sent and you received it today.)   I really want to get']

In [58]:
tokenizer.decode(generated_text[0], skip_special_tokens=True)

'Help me do hello world, sorry but I can\'t read that...\nJust check your mail! (I believe you were in "emergency mail," which means it was sent and you received it today.)   I really want to get'